# Fall 2024 Data Science Track: Week 2 - Data Cleaning Exercise

## Packages, Packages, Packages!

Import *all* the things here! You need the standard stuff: `pandas` and `numpy`.

If you got more stuff you want to use, add them here too. 🙂

In [2]:
# Import here.
import pandas as pd
import numpy as np 



## Introduction

With the packages out of the way, now you will be working with the following data sets:

* `food_coded.csv`: [Food choices](https://www.kaggle.com/datasets/borapajo/food-choices?select=food_coded.csv) from Kaggle
* `Ask A Manager Salary Survey 2021 (Responses) - Form Responses 1.tsv`: [Ask A Manager Salary Survey 2021 (Responses)](https://docs.google.com/spreadsheets/d/1IPS5dBSGtwYVbjsfbaMCYIWnOuRmJcbequohNxCyGVw/view?&gid=1625408792) as *Tab Separated Values (.tsv)* from Google Docs

Each one poses different challenges. But you’ll―of course―overcome them with what you learned in class! 😉

## Food Choices Data Set

### Load the Data

In [3]:
# Load the Food choices data set into a variable (e.g., df_food).

food_data_set_path = '../data/food_coded.csv'

df_food = pd.read_csv(food_data_set_path)

### Explore the Data

How much data did you just load?

In [4]:
# Count by hand. (lol kidding)

print(df_food.shape)
df_food.memory_usage()

(125, 61)


Index                132
GPA                 1000
Gender              1000
breakfast           1000
calories_chicken    1000
                    ... 
type_sports         1000
veggies_day         1000
vitamins            1000
waffle_calories     1000
weight              1000
Length: 62, dtype: int64

What are the columns and their types in this data set?

In [5]:
# Show the column names and their types.
print(type(df_food.columns))
print(df_food.columns)
print(df_food.dtypes)

<class 'pandas.core.indexes.base.Index'>
Index(['GPA', 'Gender', 'breakfast', 'calories_chicken', 'calories_day',
       'calories_scone', 'coffee', 'comfort_food', 'comfort_food_reasons',
       'comfort_food_reasons_coded', 'cook', 'comfort_food_reasons_coded.1',
       'cuisine', 'diet_current', 'diet_current_coded', 'drink',
       'eating_changes', 'eating_changes_coded', 'eating_changes_coded1',
       'eating_out', 'employment', 'ethnic_food', 'exercise',
       'father_education', 'father_profession', 'fav_cuisine',
       'fav_cuisine_coded', 'fav_food', 'food_childhood', 'fries', 'fruit_day',
       'grade_level', 'greek_food', 'healthy_feeling', 'healthy_meal',
       'ideal_diet', 'ideal_diet_coded', 'income', 'indian_food',
       'italian_food', 'life_rewarding', 'marital_status',
       'meals_dinner_friend', 'mother_education', 'mother_profession',
       'nutritional_check', 'on_off_campus', 'parents_cook', 'pay_meal_out',
       'persian_food', 'self_perception_weight

### Clean the Data

Perhaps we’d like to know more another day, but the team is really interested in just the relationship between calories (`calories_day`) and weight. …and maybe gender.

Can you remove the other columns?

In [6]:
# Remove ‘em.
df_cal = df_food["calories_day"]
df_weight = df_food["weight"]
df_gender = df_food["Gender"]
df = pd.concat([df_cal, df_weight, df_gender], axis=1)


What about `NaN`s? How many are there?

In [7]:
# Count ‘em. with isna()
size = len(df)
print(size)

print(df.isna().sum())




125
calories_day    19
weight           2
Gender           0
dtype: int64


We gotta remove those `NaN`s―the entire row.

In [8]:
# Drop ‘em.
#boolean exclude
val_cal = ~df["calories_day"].isna() 
val_weight = ~df["weight"].isna()
val_gend = ~df["Gender"].isna()

#alternatively, use for col in df.columns:
df_good = df[val_cal & val_weight & val_gend]

print(df_good.isna().sum())
    


calories_day    0
weight          0
Gender          0
dtype: int64


But what about the weird non-numeric values in the column obviously meant for numeric data?

Notice the data type of that column from when you got the types of all the columns?

If only we could convert the column to a numeric type and drop the rows with invalid values. 🤔

In [10]:
# Fix that.
df = df_good
for column in df.columns:
    condition = df[column]  
    df[column] = pd.to_numeric(df[column], errors='coerce')
    df = df.dropna(subset = column)
    


Now this data seems reasonably clean for our purposes! 😁

Let’s save it somewhere to be shipped off to another teammate. 💾

In [12]:
# Savey save!
df.to_csv("have fun other teammate")


## Ask a Manager Salary Survey 2021 (Responses) Data Set

### Load the Data

In [16]:
# Load the Ask A Manager Salary Survey 2021 (Responses) data set into a variable (e.g., df_salary).
df = pd.read_csv("../data/Ask A Manager Salary Survey 2021 (Responses) - Form Responses 1.tsv", sep='\t')


Was that hard? 🙃

#### rename the file to something that is better for all systems.  
* No spaces in filename (can use '_')
* all lower case

### Explore

You know the drill.

How much data did you just load?

In [20]:
# Count by hand. I’m dead serious.
print(len(df))

28062


What are the columns and their types?

In [21]:
# Show the column names and their types.
print(df.count())


Timestamp                                                                                                                                                                                                                               28062
How old are you?                                                                                                                                                                                                                        28062
What industry do you work in?                                                                                                                                                                                                           27988
Job title                                                                                                                                                                                                                               28061
If your job title needs additional context, plea

Oh… Ugh! Give these columns easier names to work with first. 🙄

In [23]:
# Rename ‘em.
# Non-binding suggestions: timestamp, age, industry, title, title_context, salary, additional_compensation, currency, other_currency, salary_context, country, state, city, total_yoe, field_yoe, highest_education_completed	gender, race
COLUMN_MAP = {
    "How old are you?": "age",
    "What industry do you work in?": "industry",
    "Job title": "job_title",
    "If your job title needs additional context, please clarify here:": "job_title_context",
    "What is your annual salary? (You'll indicate the currency in a later question. If you are part-time or hourly, please enter an annualized equivalent -- what you would earn if you worked the job 40 hours a week, 52 weeks a year.)": "annual_salary",
    "How much additional monetary compensation do you get, if any (for example, bonuses or overtime in an average year)? Please only include monetary compensation here, not the value of benefits.": "additional_compensation",
    "Please indicate the currency": "currency",
    'If "Other," please indicate the currency here:': "currency_other",
    "If your income needs additional context, please provide it here:": "income_context",
    "What country do you work in?": "country",
    "If you're in the U.S., what state do you work in?": "us_state",
    "What city do you work in?": "city",
    "How many years of professional work experience do you have overall?": "experience_years_total",
    "How many years of professional work experience do you have in your field?": "experience_years_field",
    "What is your highest level of education completed?": "education_level",
    "What is your gender?": "gender",
    "What is your race? (Choose all that apply.)": "race"
}

df = df.rename(columns=COLUMN_MAP)

print(df.columns)


Index(['Timestamp', 'age', 'industry', 'job_title', 'job_title_context',
       'annual_salary', 'additional_compensation', 'currency',
       'If "Other," please indicate the currency here: ', 'income_context',
       'country', 'us_state', 'city', 'experience_years_total',
       'experience_years_field', 'education_level', 'gender', 'race'],
      dtype='object')


It’s a lot, and that should not have been easy. 😏

You’re going to have a gander at the computing/tech subset first because thats *your* industry. But first, what value corresponds to that `industry`?

In [62]:
# List the unique industries and a count of their instances.
#industry
print(df['industry'].value_counts())
#print(df.groupby('industry').count())

industry
Computing or Tech                        4699
Education (Higher Education)             2464
Nonprofits                               2419
Health care                              1896
Government and Public Administration     1889
                                         ... 
Warehousing                                 1
Education (Early Childhood Education)       1
SAAS                                        1
Health and Safety                           1
Aerospace Manufacturing                     1
Name: count, Length: 1219, dtype: int64


That value among the top 5 is what you’re looking for innit? Filter out all the rows not in that industry and save it into a new dataframe. 

In [76]:
# Filtery filter. (Save it to a new variable, df_salary_tech.)
tech = ["Computing or Tech", "Education (Higher Education)", "Nonprofits", "Health care", "Government and Public Administration"]
df_salary_tech = df[df['industry'].isin(tech)]

Do a sanity check to make sure that the only values you kept are the one you are filtered for.  

In [77]:
# Sanity Check 

print(df_salary_tech['industry'].value_counts())



industry
Computing or Tech                       4699
Education (Higher Education)            2464
Nonprofits                              2419
Health care                             1896
Government and Public Administration    1889
Name: count, dtype: int64


We are very interested in salary figures. But how many dollars 💵 is a euro 💶 or a pound 💷? That sounds like a problem for another day. 🫠

For now, let’s just look at U.S. dollars (`'USD'`).

In [78]:
# Filtery filter for just the jobs that pay in USD!
print(df_salary_tech.columns)
usd = df['If "Other," please indicate the currency here: '] == "USD"
df_salary_tech = df_salary_tech[usd].copy()


Index(['Timestamp', 'age', 'industry', 'job_title', 'job_title_context',
       'annual_salary', 'additional_compensation', 'currency',
       'If "Other," please indicate the currency here: ', 'income_context',
       'country', 'us_state', 'city', 'experience_years_total',
       'experience_years_field', 'education_level', 'gender', 'race'],
      dtype='object')


/tmp/ipykernel_2344/1033964639.py:4: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df_salary_tech = df_salary_tech[usd].copy()


What we really want know is how each U.S. city pays in tech. What value in `country` represents the United States of America?

In [79]:
# We did filter for USD, so if we do a count of each unique country in descending count order, the relevant value(s) should show up at the top.

print(df_salary_tech["country"].value_counts())

country
USA    4
US     2
Name: count, dtype: int64


### Clean the Data

Well, we can’t get our answers with what we currently have, so you’ll have to make some changes.

Let’s not worry about anything below the first 5 values for now. Convert the top 5 to a single canonical value―say, `'US'`, which is nice and short.

In [80]:
# Replace them all with 'US'.

df_salary_tech['country'] = df_salary_tech['country'].replace({'USA': 'US'})


Have a look at the count of each unique country again now.

In [81]:
# Count again.

print(df_salary_tech["country"].value_counts())

country
US    6
Name: count, dtype: int64


Did you notice anything interesting?

In [ ]:
# BONUS CREDIT: resolve [most of] those anomalous cases too without exhaustively taking every variant literally into account.



In [ ]:

# BONUS CREDIT: if you’ve resolved it, let’s see how well you did by counting the number of instances of each unique value.



It’s looking good so far. Let’s find out the minimum, mean, and maximum (in that order) salary by state, sorted by the mean in descending order.

In [82]:
# Find the minimum, mean, and maximum salary in USD by U.S. state.
#print(df_salary_tech.copy().columns)
#df_salary_tech = df_salary_tech.copy()
df_salary_tech.groupby('us_state')['annual_salary'].agg(['min', 'mean', 'max'])


TypeError: agg function failed [how->mean,dtype->object]

Well, pooh! We forgot that `salary` isn’t numeric. Something wrong must be fixed.

In [83]:
# Fix it.
df_salary_tech['annual_salary'] = pd.to_numeric(df_salary_tech['annual_salary'], errors="coerce")
df_salary_tech = df_salary_tech.dropna(subset=['annual_salary'])
print(df_salary_tech.columns)
#df_salary_tech.groupby('us_state')['annual_salary'].agg(['min', 'mean', 'max'])


Index(['Timestamp', 'age', 'industry', 'job_title', 'job_title_context',
       'annual_salary', 'additional_compensation', 'currency',
       'If "Other," please indicate the currency here: ', 'income_context',
       'country', 'us_state', 'city', 'experience_years_total',
       'experience_years_field', 'education_level', 'gender', 'race'],
      dtype='object')


Let’s try that again.

In [84]:
# Try it again. Yeah!

df_salary_tech.groupby('us_state')['annual_salary'].agg(['min', 'mean', 'max'])

,min,mean,max
us_state,,,
California,99000.0,99000.0,99000.0
Illinois,85000.0,85000.0,85000.0


That did the trick! Now let’s narrow this to data 2021 and 2022 just because (lel). *(Hint: that timestamp column may not be a temporal type right now.)*

In [85]:
# Filter the data to within 2021, 2022, or 2023, saving the DataFrame to a new variable, and generate the summary again.

df_salary_tech['Timestamp'] = pd.to_datetime(df_salary_tech['Timestamp'], errors='coerce')

df_filtered = df_salary_tech[df_salary_tech['Timestamp'].dt.year.isin([2021, 2022])]

print(df_filtered)


                Timestamp    age                              industry  \
21499 2021-04-30 10:39:24  45-54                           Health care   
25539 2021-05-06 19:56:12  55-64  Government and Public Administration   

                       job_title job_title_context  annual_salary  \
21499             DBA/Programmer               NaN        85000.0   
25539  Administrative Analyst II               NaN        99000.0   

       additional_compensation currency  \
21499                      0.0    Other   
25539                      0.0    Other   

      If "Other," please indicate the currency here:  income_context country  \
21499                                             USD            NaN      US   
25539                                             USD            NaN      US   

         us_state          city experience_years_total experience_years_field  \
21499    Illinois  Collinsville          21 - 30 years          11 - 20 years   
25539  California       Oakland     

/tmp/ipykernel_2344/3114660916.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_salary_tech['Timestamp'] = pd.to_datetime(df_salary_tech['Timestamp'], errors='coerce')


## Bonus

Clearly, we do not have enough data to produce useful figures for the level of specificity you’ve now reached. What do you notice about Delaware and West Virginia?

Let’s back out a bit and return to `df_salary` (which was the loaded data with renamed columns but *sans* filtering).

### Bonus #0

Apply the same steps as before to `df_salary`, but do not filter for any specific industry. Do perform the other data cleaning stuff, and get to a point where you can generate the minimum, mean, and maximum by state.

### Bonus #1

This time, format the table output nicely (*$12,345.00*) without modifying the values in the `DataFrame`. That is, `df_salary` should be identical before versus after running your code.

(*Hint: if you run into an error about `jinja2` perhaps you need to `pip install` something.*)

### Bonus #2

Filter out the non-single-states (e.g., `'California, Colorado'`) in the most elegant way possible (i.e., *not* by blacklisting all the bad values).

### Bonus #3

Show the quantiles instead of just minimum, mean, and maximum―say 0%, 5%, 25%, 50%, 75%, 95%, and 100%. Outliers may be deceiving.

Sort by whatever interests you―like say the *50th* percentile.

And throw in a count by state too. It would be interesting to know how many data points contribute to the figures for each state. (*Hint: your nice formatting from Bonus #1 might not work this time around.* 😜)